<a href="https://www.hydroffice.org/openbst/"><img src="images/openbst.png" alt="XSF" title="Open OpenBST home page" align="center" width="12%" alt="XSF logo"></a>

<center><h1>OpenBST: An interactive and open library for Backscatter Processing </h1></center>

***OpenBST:*** The <strong>Open</strong>  <strong>B</strong>ack <strong>S</strong>catter <strong>T</strong> oolchain project is an open and collaborative effort that provides an interactive environment to explore backscatter data and processing algorithms. The project is designed as a virtual town square where developers, researchers and users can come to discuss the latest state of the art in backscatter processing algorithms.

# Introduction to Jupyter Notebooks

OpenBST uses Jupyter Notebooks as the interactive front end. There are two types of cells used in the notebook:

### Markdown Cells:
This is a markdown cell. It contains texual information and directions for interacting with the notebook and project. An empty markdown cell is shown below. Type something and then run the cell. You can view the markup text buy pressing `enter`

*hint: to run a cell, press `shift + enter`. This will run the cell and select the next cell*

### Code Cells:
Code cells let you run python commands. The real advantage of the Notebooks is that as you move through the processing workflow, you are able to interact with the public API and test various functionalities and methodologies. Below is a code cell with a bit of code. Feel free to test out other python commands.


In [16]:
print("Hola Mundo")

Hola Mundo


There is plenty more about Jupyter Notebooks to learn and they really are a neat way of programming. But for this project you now know all you need to know about Notebooks. Now lets get Started!

# Running the Notebook

At this point we can now start to interact with a standard notebook. Each section interacts with the public API that runs in the background. The only cell that needs to be run first is the "Setup the Notebook Environment". After that you're free to skip cells. This is because we don't want to impose an order of operations to backscatter processing unless everyone agrees there should be one. Of course, there are sections that may need information from other sections. If that's the case, the code will tell you what you need to run.

Have at it!

## Setup the Notebook Environment

Before starting, you must execute the cell below. This code sets up the namespace of the notebook.

In [17]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display

from hyo2.openbst.lib.openbst import OpenBST

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create an OpenBST Project

One of the first things we need to do is create a project. Interacting with the project lets you add files, test a processing algorithm, and visualize the results. 

By default, project files are placed in the local AppData directory. When creating a new project make sure to use a unique name. Each notebook will have a suggested project name, but you are the boss so make it your own.

If you want to continue with an old project, enter the old project title to open that project where you left off.

In [19]:
project_title = "getting_started"
bst = OpenBST(prj_name=project_title).prj
print("Project Path: " + str(bst.info.project_path))
print("Project Version: " + str(bst.info.version))
print("Last Modified: " + str(bst.info.modified) + " UTC")
bst.open_project_folder()

Project Path: C:\Users\msmith\AppData\Local\HydrOffice\OpenBST\projects\getting_started.openbst
Project Version: 0.1.1
Last Modified: 2019-11-20 21:32:29.400157 UTC


The code above should have opened the your directory `prj_name.openbst`. This is where all the project files are stored. 
Take a moment look over the directory structure:

### Project Directories:

If you are looking at your project structure you should see three directories. Below is a short description of what gets stored in each of these:


* **Raws** - When a new raw file (.all, .s7k, etc.) is added to the project, a copy is made and stored at a netCDF file


* **Processing** - For each data file, a netCDF file is created to store the results of a processing step and the processing metadata


* **Products** - This folder contains netCDF files which store and recall the data visualizations generated at the processing steps. 

### NetCDF Files:

You will notice that the main data storage contained mentioned above is a netCDF file. Additionally you should be able to see the `info.nc` file in the project directory. `info.nc` is a netcdf file used to maintain information about the project such as what files have been added or removed, processed, etc.

netCDF files are well equipped to handle sonar data, and the CF convention allows for easy metadata coupling. Additionally, the CF convention is well known and data exploration is possible with other computing environments (c++, python, Matlab) and existing softwares (Arc, QGIS). This is a major benefit if you want to take the processing results and inspect it with your prefered method.

A free an convenient application to explore the data is NASA's [Panoply](https://www.giss.nasa.gov/tools/panoply/)

## Add Files

Use the widget below to add files to the project. You can add sonar files at any time. However, adding files in the middle of processing requires all steps up to that point. You can add as many files as you like.

As the title of the notebooks says, this notebook is focused on the T50-P. We will yell at you if load data from different systems. So do not try anything funny!

In [20]:
# Widgets are not in the MVP at the moment. But will be added as time permits. For now the cell below will suffice for testing

In [21]:
from hyo2.abc.lib.testing_paths import TestingPaths
testing = TestingPaths(root_folder=Path().cwd().parents[2].resolve())
print("Root Testing Path: " + str(testing.root_folder))
raw_path = testing.download_data_folder().joinpath('reson','20190730_144835.s7k')
print("Raw File Path: " + str(raw_path))

Root Testing Path: C:\PythonCode\hyo2_openbst
Raw File Path: C:\PythonCode\hyo2_openbst\data\download\reson\20190730_144835.s7k


In [22]:
added = bst.add_raw(raw_path)

## Check Project Health

Before processing, the project checks to make sure that it is in a healthy state. A healthy project is a happy project.  
This means checking that all the raw files have been imported into the project, there are no broken files or anything missing.


After running this section you should be able to explore the raw data file created in the raws folder.

In [23]:
bst.check_health()

# Processing

We will now walk through the processing workflow. Each subsection will conduct some processing action that is part of the backscatter processing toolchain. At each step, a number of methodologies can be utilized and compared. 

In [ ]:

# Open Data File and Map the datagrams
infile = prr.x7kRead(str(raw_input))
infile.mapfile(verbose=False)

# Extract data from the datafile
# - Get the sonar setting data
dg_type = '7000'
nr_dg_runtime = len(infile.map.packdir[dg_type])
# 39 is the number of fields in the runtime datagram
data_runtime = np.empty((nr_dg_runtime, 39), dtype=float)
for i in range(nr_dg_runtime):
    try:
        dg = infile.getrecord(dg_type, i)
        # this cast is dangerous
        data_runtime[i, :] = np.asarray(dg.header, dtype=float)

    except Exception:
        logger.error(traceback.format_exc())
        logger.error("Error while reading %s #%d" % (dg_type, i))

# - Get the receiver beam widths
dg_type = '7004'
nr_dg_beamgeo = len(infile.map.packdir[dg_type])
# -- determine how many rx beams system was set to. Assume constant through dg
dg = infile.getrecord(dg_type, 1)
nr_rx_beams = dg.header[1]
# -- reading all the 4 fields in the data section
data_beamgeo = np.empty((nr_dg_beamgeo, nr_rx_beams, 4))
for i in range(nr_dg_beamgeo):
    try:
        dg = infile.getrecord(dg_type, i)
        data_beamgeo[i, :, :] = dg.data.transpose()
    except Exception:
        logger.error(traceback.format_exc())
        logger.error("Error while reading %s #%d" % (dg_type, i))

# - Get the sonar bathy/Intensity data
dg_type = '7027'
nr_pings = len(infile.map.packdir[dg_type])

data_bathy = np.empty((nr_pings, nr_rx_beams, 7))  # 7 because of selecting some of the fields
data_bathy[:] = np.nan
time_bathy = np.empty(nr_pings)
time_bathy[:] = np.nan
for i in range(nr_pings):
    try:
        time_bathy[i] = infile.map.packdir[dg_type][i, 1]
        dg = infile.getrecord(dg_type, i)
        beam_indices = dg.data[:, 0].astype(int)
        data_bathy[i, beam_indices, 1:4] = dg.data[:, [1, 2, 6]]
        # change to avoid expensive matrix multiplication
        data_bathy[i, :, 0] = np.tile(np.array(dg.header[1], copy=True), nr_rx_beams)
        data_bathy[i, :, 4] = np.tile(np.array(dg.header[7], copy=True), nr_rx_beams)
        data_bathy[i, :, 5] = np.tile(np.array(dg.header[8], copy=True), nr_rx_beams)
        data_bathy[i, :, 6] = np.tile(np.array(dg.header[9], copy=True), nr_rx_beams)

    except Exception:
        logger.error(traceback.format_exc())
        logger.error("Error while reading %s #%d" % (dg_type, i))

# - Get the TVG
dg_type = '7010'
nr_dg_tvg = len(infile.map.packdir[dg_type])
data_tvg = list()
for i in range(nr_dg_tvg):
    try:
        dg = infile.getrecord(dg_type, i)
        data_tvg.append(dg.data)
    except Exception:
        logger.error(traceback.format_exc())
        logger.error("Error reading %s #%d" % (dg_type, i))

# - Get the position data
dg_type = '1003'
nr_dg_position = len(infile.map.packdir[dg_type])
time_position = np.empty(nr_dg_position)
position_measured = np.empty((nr_dg_position, 3))  # 3 because we read just 3 of the fields
for i in range(nr_dg_position):
    try:
        time_position[i] = infile.map.packdir[dg_type][i, 1]
        dg = infile.getrecord(dg_type, i)
        if dg.header[0] != 0:
            logger.debug("Warning: Datum is not WGS84")
        lat_rad, lon_rad = dg.header[2:4]
        height_datum = dg.header[4]
        lat_degree = np.rad2deg(lat_rad)
        lon_degree = np.rad2deg(lon_rad)
        position_measured[i, :] = [lat_degree, lon_degree, height_datum]
    except Exception:
        logger.error(traceback.format_exc())
        logger.error("Error reading %s #%d" % (dg_type, i))
# -- Interpolate the position to the ping time
lat_interp_object = interp_sp.interp1d(time_position, position_measured[:, 0], fill_value='extrapolate')
lat_ping = lat_interp_object(time_bathy)

lon_interp_object = interp_sp.interp1d(time_position, position_measured[:, 1], fill_value='extrapolate')
lon_ping = lon_interp_object(time_bathy)

# height_interp_object = interp_sp.interp1d(time_position, position_measured[:, 2], fill_value='extrapolate')
# height_ping = height_interp_object(time_bathy)

# data_position = np.array([lat_ping, lon_ping, height_ping])
# data_position = data_position.transpose()

# - Get the attitude data
# -- Get the roll, pitch, heave / Values are in radians
dg_type = '1012'
nr_dg_rph = len(infile.map.packdir[dg_type])
time_rph = np.empty(nr_dg_rph)
rph_measured = np.empty((nr_dg_rph, 3))
for i in range(nr_dg_rph):
    try:
        time_rph[i] = infile.map.packdir[dg_type][i, 1]
        dg = infile.getrecord(dg_type, i)
        rph_measured[i, :] = dg.header

    except Exception:
        logger.error(traceback.format_exc())
        logger.error("Error reading %s #%d" % (dg_type, i))

# -- Get the heading
dg_type = '1013'
nr_dg_heading = len(infile.map.packdir[dg_type])
time_heading = np.empty(nr_dg_heading)
heading_measured = np.empty(nr_dg_heading)
for i in range(nr_dg_heading):
    try:
        time_heading[i] = infile.map.packdir[dg_type][i, 1]
        dg = infile.getrecord(dg_type, i)
        heading_measured[i] = np.array(dg.header)

    except Exception:
        logger.error(traceback.format_exc())
        logger.error("Error reading %s #%d" % (dg_type, i))

# --Interpolate the attitude to the ping times
# roll_interp_object = interp_sp.interp1d(time_rph, rph_measured[:, 0], fill_value='extrapolate')
# roll_ping = roll_interp_object(time_bathy)
#
# pitch_interp_object = interp_sp.interp1d(time_rph, rph_measured[:, 1], fill_value='extrapolate')
# pitch_ping = pitch_interp_object(time_bathy)
#
# heave_interp_object = interp_sp.interp1d(time_rph, rph_measured[:, 2], fill_value='extrapolate')
# heave_ping = heave_interp_object(time_bathy)

# TODO: Determine if extrapolate is good or bad
heading_interp_object = interp_sp.interp1d(time_heading, heading_measured[:], fill_value='extrapolate')
heading_ping = heading_interp_object(time_bathy)

In [ ]:
 # Processing Workflow
    # - Convert digital value to dB
    digital_value_db = 20 * np.log10(data_bathy[:, :, 3])

    # -- Plot the per beam reflectivity
    frequency = data_runtime[0, 3]
    title_str = "Initiial raw Relfectivity\n Reson T50-P @ %d kHz" % (frequency / 1000)
    fig_raw = data_plt.plot_ping_beam(digital_value_db, title=title_str)

In [ ]:

    # - Calculate the estimated slant range
    bottom_detect_sample = data_bathy[:, :, 1]
    surface_sound_speed = data_runtime[:, 36]
    sample_rate = data_bathy[:, :, 4]
    range_m = bottom_detect_sample / sample_rate * surface_sound_speed[:, np.newaxis] / 2

    # -- Plot the range and check it's within reason
    cmap = 'gist_rainbow'
    title_str = "Preliminary Slant Range"
    clabel_str = "Range [m]"
    fig_range = data_plt.plot_ping_beam(range_m, colormap=cmap, title=title_str, clabel=clabel_str)


In [ ]:
# - Remove the static gain
    rx_fixed_gain = data_runtime[:, 15]
    datacorr_fixed_gain = digital_value_db - rx_fixed_gain[:, np.newaxis]

    # -- Plot the adjusted gain
    title_str = "Static Gain [%.1f dB] Correction Product\nReson T50-P @ %d kHz" % (rx_fixed_gain[0], frequency/1000)
    fig_fixedgain = data_plt.plot_ping_beam(datacorr_fixed_gain, title=title_str)

In [ ]:
# - Remove the TVG
    tvg_gain = np.empty((nr_dg_tvg, nr_rx_beams))

    for i in range(nr_dg_tvg):  # TODO: Make something better than a nested for loop
        for j in range(nr_rx_beams):
            if np.isnan(bottom_detect_sample[i, j]):
                tvg_gain[i, j] = np.nan
            else:
                # TODO: verify rounding logic
                tvg_index = int(np.round(bottom_detect_sample[i, j]))
                tvg_curve = data_tvg[i]
                tvg_gain[i, j] = tvg_curve[tvg_index]

    alpha = data_runtime[:, 35] / 1000
    spreading = data_runtime[:, 37]
    tvg_bswg = (spreading[:, np.newaxis] * np.log10(range_m)) + 2 * (alpha[:, np.newaxis] * range_m)
    datacorr_tvg_gain = digital_value_db - tvg_gain

    # -- Plot a comparison between the calculated and estimated tvg values
    mplt.figure()
    mplt.plot(np.nanmean(tvg_gain, axis=0))
    mplt.plot(np.nanmean(tvg_bswg, axis=0))
    mplt.grid()
    mplt.title(
        "Comparison between RESON TVG and BSWG TVG\nSpreading = %d dB / Absorption = %0.2f dB/m"
        % (spreading[0], alpha[0]))
    mplt.xlabel("Beam [#]")
    mplt.ylabel("Average TVG value [dB}")
    mplt.legend(["Reson TVG", "BSWG TVG"])

    # -- Plot the corrected values using the reson tvg
    title_str = "TVG Correction Product\nReson T50-P @ %dHz" % (frequency/1000)
    fig_tvgcorr = data_plt.plot_ping_beam(datacorr_tvg_gain, title=title_str)
    mplt.imshow(datacorr_tvg_gain, cmap='Greys_r')

In [ ]:
# - Correct for the Source Level
source_level = data_runtime[:, 14]
datacorr_sourcelevel = datacorr_tvg_gain - source_level[:, np.newaxis]

# -- Plot the corrected values
title_str = "Source Level Correction Product\nReson T50-P @ %dkHz" % (frequency/1000)
fig_sourcelevel = data_plt.plot_ping_beam(datacorr_sourcelevel, title=title_str)


In [ ]:
# - Apply Relative Calibration Correction
# -- Load calibration curve
calibration_data = np.genfromtxt(fname=calib_input, dtype=float, delimiter=',')

# -- Generate a 4th Order Fit to the curve data
poly_coefficents = np.polyfit(calibration_data[:, 0], calibration_data[:, 1], 4)
calibration_curve = np.arange(-75, 75, 0.1)[:, np.newaxis]
calibration_curve = np.tile(calibration_curve, [1, 2])
calibration_curve[:, 1] = np.polyval(poly_coefficents, calibration_curve[:, 0])

# -- Obtain Calibration Correction for each beam angle of each ping
rx_angle = data_bathy[:, :, 2]
calibration_correction = np.empty((nr_pings, nr_rx_beams))
for i in range(nr_pings):
    calibration_correction[i, :] = np.interp(np.rad2deg(rx_angle[i, :]),
                                             calibration_curve[:, 0], calibration_curve[:, 1])
# -- Apply calibration values
datacorr_echolevel = datacorr_sourcelevel - calibration_correction

# -- Plot the corrected values
title_str = "Echo Level Product\nReson T50-P @ %dkHz" % (frequency/1000)
clabel_str = "Intensity Value [dB re 1$mu$Pa]"
fig_echo = data_plt.plot_ping_beam(datacorr_echolevel, title=title_str, clabel=clabel_str)


In [ ]:
# - Correct for the Transmission Loss
# TODO: current assumption is spherical spreading
transmission_loss = 40 * np.log10(range_m) + 2 * (alpha[:, np.newaxis] / 1000) * range_m
datacorr_transmissionloss = datacorr_sourcelevel + transmission_loss

# -- Plot the corrected values
title_str = "Transmission Loss Correction Product\nReson T50-P @ %dkHz" % (frequency/1000)
clabel_str = "Intensity Value [dB re 1$mu$Pa]"
fig_transmission = data_plt.plot_ping_beam(datacorr_transmissionloss, title=title_str, clabel=clabel_str)

In [ ]:
# - Correct for Ensonified Area
    # -- Calculate the Ensonified area using a flat seafloor assumption
    beamwidth_rx_across = data_beamgeo[:, :, 3]
    beamwidth_tx_along = data_runtime[:, 20]
    pulse_length = data_runtime[:, 6]
    ind_zero = np.where(rx_angle == 0)
    rx_angle[ind_zero[0], ind_zero[1]] = np.deg2rad(0.1)  # Nudge 0 values
    area_beamlimited = beamwidth_rx_across * beamwidth_tx_along[:,
                                             np.newaxis] * range_m ** 2  # TODO: Verify that the 7004 dg is accounting for the cosine of the angle
    area_pulselimited = ((surface_sound_speed[:, np.newaxis] * pulse_length[:, np.newaxis]) / (
            2 * np.sin(np.abs(rx_angle)))) \
                        * (beamwidth_tx_along[:, np.newaxis] * range_m)

    # TODO: justify why taking the minimum
    area_correction = 10 * np.log10(np.minimum(area_beamlimited, area_pulselimited))

    # -- Correct the data
    datacorr_radiometric = datacorr_transmissionloss - area_correction

    # -- Plot area regions
    fig_areacorr = mplt.figure()
    mplt.plot(10 * np.log10(np.nanmean(area_beamlimited, axis=0)))
    mplt.plot(10 * np.log10(np.nanmean(area_pulselimited, axis=0)))
    mplt.plot(10 * np.log10(np.nanmean(np.minimum(area_pulselimited, area_beamlimited), axis=0)))
    mplt.grid(which='minor')
    mplt.xlabel("Beam [#]")
    mplt.ylabel("Area Correction [dB]")
    mplt.title("Comparison of Area Corrections")

    # Plot the area corrected data
    title_str = "Seafloor Backscatter Product\nReson T50-P @ %dkHz" % (frequency/1000)
    clabel_str = r'$S(\theta)_b$  [dB re $1\mu$Pa]'
    fig_radiometric = data_plt.plot_ping_beam(datacorr_radiometric, title=title_str, clabel=clabel_str)